# トークナイザの作成

ペアデータを元に作成する

In [2]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordPiece
from tokenizers.pre_tokenizers import WhitespaceSplit
import tokenizers.decoders as decoders
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.decoer = decoders.BPEDecoder()

# tokenizer = Tokenizer(Unigram())
# tokenizer = Tokenizer(WordPiece(unk_token="<unk>",))
# tokenizer.decoder = decoders.WordPiece()

tokenizer.pre_tokenizer = WhitespaceSplit()

In [3]:
vocab_size = 5000
special_tokens = ["<s>", "<pad>", "</s>", "<unk>"]
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(vocab_size=vocab_size, special_tokens=special_tokens, show_progress=True)
# from tokenizers.trainers import UnigramTrainer
# trainer = UnigramTrainer(special_tokens=special_tokens, show_progress=True)
# from tokenizers.trainers import WordPieceTrainer
# trainer = WordPieceTrainer(special_tokens=special_tokens, show_progress=True)

In [4]:
def gen():
    import json
    data = json.load(open("all.json", "r"))
    for d in data:
        yield d["text"]
        yield d["phoneme_text"]

In [5]:
tokenizer.train_from_iterator(gen(), trainer=trainer)

In [6]:
from tokenizers.processors import TemplateProcessing

tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> </s> <s> $B </s>",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ],)

In [7]:
tokenizer.save("tokenizer.json")


In [9]:
text = "それではこれから発表を始めたいと思います"
encoded = tokenizer.encode(text)
tokenizer.decode(encoded.ids)

'それ では これから 発表 を 始め たい と思います'

In [11]:
text = "^ h a [ cl py o o # sh i [ m a ] s u $"
encoded = tokenizer.encode(text)
print(encoded.ids)
tokenizer.decode(encoded.ids)

[0, 11, 20, 13, 9, 3303, 3575, 27, 27, 4, 3302, 21, 9, 25, 13, 10, 30, 32, 5, 2]


'^ h a [ cl py o o # sh i [ m a ] s u $'